## US Census Bureau API

In [11]:
import pandas as pd 
import json
import requests
import os 
from pprint import pprint
import glob

pd.set_option('display.float_format', '{:.5f}'.format)
pd.set_option('display.max_columns', None)

In [3]:
api_key = open('/Users/admin/Desktop/opioid_project/census_api_key.txt').read().strip()

### Getting the population from every state, every year from 2009 to 2019

In [8]:
totalDF = []
# even though the opioid dataset is from 2006 to 2019, the census bureau's variable for population doesn't go back past 2009 so I need to start from that year
yearList = pd.Series(range(2009,2020)).to_list()
yearList
variablesList = 'B01001_001E'

for year in yearList:
    url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME,{variablesList}&for=state:*&key={api_key}'
    data = requests.get(url)
    # the output is a list of lists in which the first lists is the header values / columns, and subsequent values in the list are rows
    data = data.json()
    
    # turning the output into a dataframe
    df = pd.DataFrame(data)
    # turning the first row into columns
    df.columns = df.iloc[0]
    # since the first row is now the columns / header, I need rows starting at [1]
    df = df[1:]
    
    df['year'] = year
    
    totalDF.append(df)


df = pd.concat(totalDF, axis = 0)

In [9]:
df = df.drop(columns='state', axis = 1)
df = df[df['NAME'] != 'Puerto Rico']

In [33]:
df = df.rename(columns= 
                {'B01001_001E': 'population',
                 'NAME': 'state'
                })
df.to_csv('census.csv', index=False)

### Reading all the csv's and using certain columns so it'll be smaller when I import to PG

In [ ]:
file_path = '/Users/admin/Desktop/states/arcos-*-statewide-itemized.csv'
files = glob.glob(file_path)

In [ ]:
for file in files:
    df = pd.read_csv(file, usecols= ['TRANSACTION_DATE',
                                     'REPORTER_DEA_NO',
                                     'REPORTER_BUS_ACT',
                                     'REPORTER_NAME',
                                     'REPORTER_COUNTY',
                                     'REPORTER_STATE',
                                     'BUYER_DEA_NO',
                                     'BUYER_BUS_ACT',
                                     'BUYER_NAME',
                                     'BUYER_COUNTY',
                                     'BUYER_STATE',
                                     'DRUG_NAME',
                                     'Combined_Labeler_Name',
                                     'Reporter_family',
                                     'CALC_BASE_WT_IN_GM',
                                     'DOSAGE_UNIT'])
    
    # using split because all file names share the same pattern, and I can split with the hyphen and get the second value from splitting, which is the state name abbreviation
    state_abbreviation = file.split('-')[1]
    state_name = f'{state_abbreviation}.csv'
    
    # I want separate csv's and will be appending in Postgres
    df.to_csv(state_name, index= False)
    
    print(f"{state_abbreviation} and {state_name} is done")